<div align='center'><font size='6'>CS304_Project1</font></div>


<div align='center'>Yiwei Liang, Yuchen Song</div>

# Part I

This code is designed to capture real-time speech audio using the pyaudio library, process the captured data with numpy, and display the audio using the IPython library.

In [1]:
#Use pyaudio for speech capture and use numpy for processing data
import pyaudio
import numpy as np

#Initialize parameters of pyaudio speech capture
CHUNK = 1024
resolution = pyaudio.paInt16 
CHANNELS = 1 
sampling_rate = 16000
record_time=5
Expected_num_of_chunks= int(sampling_rate/CHUNK*record_time) 
p=pyaudio.PyAudio()
stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
frames=[]

#Get data of chunks
for i in range(0,Expected_num_of_chunks): 
    data = stream.read(CHUNK)
    frames.append(data)      
array_frames=np.array(frames)

#Convert the hexadecimal data to decimal one
intframes=np.frombuffer(array_frames, np.int16)

#Stop speech capture
stream.stop_stream()
stream.close()
p.terminate()

#display the captured speech signals using IPython package
import IPython.display as ipd
ipd.Audio(intframes,rate=16000)

# Part II

The program waits for a keypress ('s') and then records audio until there is a sufficiently long period of silence, saving the recorded speech as a WAV file.

In [1]:
import keyboard
import time
import wave
import pyaudio
import numpy as np
import IPython.display as ipd
frames=[]
#Define hit-to-talk function and set 's' as the start key
def hit_to_talk():
    keyboard.wait('s')
    time.sleep(2)
    print("Please talk")

#Define the function to calculate the energy of samples in decibel
def EnergyPerSampleInDecibel(audioframe):
    audioframe=np.frombuffer(np.array(audioframe),np.int16)
    np_audioframe=np.array(audioframe,dtype='int64')
    energy= 10*np.log10(np.sum(np.square(np_audioframe)))
    return energy

#Define the function to calculate the initial backgorund energy 
def find_ini_background(audioframe):
    background_energy=0
    background_energy+=EnergyPerSampleInDecibel(audioframe)
    return background_energy/10

#Define the calssifyFrame function to find the endpointing
def classifyFrame(audioframe,background,level, First_run):
    forgetfactor=2
    isSpeech=False
    current = EnergyPerSampleInDecibel(audioframe)
    if First_run:
        First_run=False
        level=EnergyPerSampleInDecibel(audioframe)
    else:
        level=((level*forgetfactor)+current)/(1+forgetfactor)
    threshold=10
    adjustment=0.05
    print('Background',background,'current',current, 'level', level)
    if current < background:
        background = current
    else:
        background+=(current - background)*adjustment
    if (level < background): level = background
    print(level-background)
    if (level - background > threshold): isSpeech = True
    return background,level,isSpeech

def record(frames):
    #parameters for recording
    CHUNK = 1600
    resolution = pyaudio.paInt16
    CHANNELS = 1
    sampling_rate = 16000
    stop_record_time=2
    stop_speak_time=0
    stop_speak_per_loop=0.1
    isSpeech=False
    p=pyaudio.PyAudio()
    stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
    
    level=0
    First_run=True
    frames_to_store=[]
    inspeech=False
    background=0
    First_inspeech=0
    
    while True:
        #If it is the first run, read the data of 10 chunks to calculate the average energy of these 10 chunks and set it as the initial value of background
        if First_run:
            audioframe=stream.read(CHUNK*10)
            background=find_ini_background(audioframe)
            First_run=False
        #Otherwise, read the data of 1 chunk to calculate energy in the classifyFrame function
        else: 
            audioframe=stream.read(CHUNK)
        #Save the data of each chunk in the list 'frames'
            
            background,level,isSpeech=classifyFrame(audioframe,background,level,First_run)
        print(isSpeech)
        if isSpeech:
            if First_inspeech==0:
                First_inspeech=1
                frames_to_store=[]
            if frames_to_store!=[] and First_inspeech==1:
                for frame in frames_to_store:
                    frames.append(frame)
                    frames_to_store=[]
            frames.append(audioframe)
            stop_speak_time=0
        #level,isSpeech=classifyFrame(audioframe,background,level,First_run)
        #After the first iteration set First_run as False
        else: 
            if stop_speak_time<stop_record_time:
                frames_to_store.append(audioframe)
            stop_speak_time+=0.1
                
        
        #if the duration for which the energy remains below the threshold is too long, stop recording
        if stop_speak_time>=stop_record_time:
            #Save the speech to a wav file
            wf = wave.open('1.wav', 'wb')
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
            wf.setframerate(sampling_rate)
            wf.writeframes(b''.join(frames))
            wf.close()
            print("Audio data saved as 1.wav")
            stream.stop_stream()
            stream.close()
            p.terminate()
            break
    
def start():
    hit_to_talk()
    record(frames)
    

In [2]:
a=start()

Please talk
False
Background 7.258223188843179 current 60.11164715276766 level 20.03721571758922
10.136321330549816
True
Background 9.900894387039404 current 65.28581470335381 level 35.120082046177416
22.44994164332229
True
Background 12.670140402855125 current 63.548069758852165 level 44.59607795040233
29.382041079747353
True
Background 15.214036870654978 current 62.39964804006263 level 50.530601313622434
32.95728388449707
True
Background 17.57331742912536 current 57.1472614499444 level 52.73615469239642
33.18414006223011
True
Background 19.552014630166312 current 58.030827509208116 level 54.50104563133365
33.02509035721525
True
Background 21.4759552741184 current 64.0950097972551 level 57.6990336866408
34.09212568636556
True
Background 23.606908000275237 current 106.307372592995 level 73.9018133220922
46.15988209218097
True
Background 27.741931229911224 current 109.16206182399253 level 85.65522948939231
53.842291729777024
True
Background 31.812937759615288 current 99.82087869270659 l